SOP0125 - Delete Key For Encryption At Rest
===========================================

Description
-----------

Use this notebook to connect to the `controller` database and delete a
key from controller database. For restoring keys with names which exist
in the target deployment, the key with the conflicting name needs to be
deleted before it can be restored. This notebook can be used to delete
keys with conflicting names.

Steps
-----

### Parameters

Set the `key_to_be_deleted` to the key name to be deleted from Big Data
Cluster.

In [ ]:
key_to_be_deleted = "your_key_name_here"
print(f"Key with name '{key_to_be_deleted}' will be deleted.")

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream
except ImportError: 

    # Install the Kubernetes module
    import sys
    !{sys.executable} -m pip install kubernetes    
    
    try:
        from kubernetes import client, config
        from kubernetes.stream import stream
    except ImportError:
        display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
        raise

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    config.load_incluster_config()
else:
    try:
        config.load_kube_config()
    except:
        display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
        raise

api = client.CoreV1Api()

print('Kubernetes client instantiated')

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Python function queries `controller` database and return results.

### Create helper function to run `sqlcmd` against the controller database

In [ ]:
try:
  import pandas
except ModuleNotFoundError:
  !{sys.executable} -m pip install --user pandas
  import pandas
from io import StringIO
pandas.set_option('display.max_colwidth', -1)
name = 'controldb-0'
container = 'mssql-server'

def run_sqlcmd(query):
    command=f"""export SQLCMDPASSWORD=$(cat /var/run/secrets/credentials/mssql-sa-password/password);
    /opt/mssql-tools/bin/sqlcmd -b -S . -U sa -Q "SET NOCOUNT ON;
    {query}" -d controller  -s"^" -W > /tmp/out.csv; sed -i 2d /tmp/out.csv;
    cat /tmp/out.csv"""
    output=stream(api.connect_get_namespaced_pod_exec, name, namespace, command=['/bin/sh', '-c', command], container=container, stderr=True, stdout=True)
    return str(output)
print("Function defined")

### Delete a key

The following cell will delete a key for HDFS encryption at rest. Once
the key is deleted, it cannot be undone. If the provided key name for
deletion exists, it will be deleted but if the key doesn’t exist, then
no operation will be performed.

In [ ]:
select_query = """ select count(*) from Credentials 
                where ISJSON(application_metadata) > 0 and JSON_VALUE(application_metadata, '$.vaultname') in ('hdfsvault-svc', 'mssqlvault-svc', 'controllervault-svc')
                and type in ('2', '3') and account_name = '{key_name_for_deletion}' """.format(key_name_for_deletion=key_to_be_deleted)
result = run_sqlcmd(select_query)

if(result == "0"):
    print(f"Key name not exists: {key_to_be_deleted}")
else:
    delete_query = """delete from Credentials 
                  where ISJSON(application_metadata) > 0 and JSON_VALUE(application_metadata, '$.vaultname') in ('hdfsvault-svc', 'mssqlvault-svc', 'controllervault-svc')
                  and type in ('2', '3') and account_name = '{key_name_for_deletion}' """.format(key_name_for_deletion=key_to_be_deleted)
    run_sqlcmd(delete_query)
    print("Key with name {keyToBeDeleted} has been deleted.".format(keyToBeDeleted = key_to_be_deleted));

print("Delete Key(s) Operation Completed.")

In [ ]:
print("Notebook execution is complete.")